In [1]:
# import os

In [3]:
# root_path = 'E:\Data\CCF_Data\Hemangioma'

# for dirname, _, filenames in os.walk(root_path):
#     print(dirname)

E:\Data\CCF_Data\Hemangioma
E:\Data\CCF_Data\Hemangioma\.checkpoint
E:\Data\CCF_Data\Hemangioma\Log
E:\Data\CCF_Data\Hemangioma\test
E:\Data\CCF_Data\Hemangioma\train
E:\Data\CCF_Data\Hemangioma\train\img
E:\Data\CCF_Data\Hemangioma\train\mask
E:\Data\CCF_Data\Hemangioma\工具类


In [44]:
# import cv2
# img = cv2.imread('E:\\Data\\CCF_Data\\Hemangioma\\train\\img\\20.png')
# print(img.shape)
# img = cv2.imread('E:\\Data\\CCF_Data\\Hemangioma\\train\\mask\\20.png')
# print(img.shape)

(299, 440, 3)
(1080, 1920, 3)


In [1]:
import torch.utils.data as data
import PIL.Image as Image
import os


def make_dataset(root):
    imgs = []
    n = len(os.listdir(root)) // 2
    for i in range(n):
        img = os.path.join(root, "%03d.png" % i)
        mask = os.path.join(root, "%03d_mask.png" % i)
        imgs.append((img, mask))
    return imgs


class LiverDataset(data.Dataset):
    def __init__(self, root, transform=None, target_transform=None):
        imgs = make_dataset(root)
        self.imgs = imgs
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        x_path, y_path = self.imgs[index]
        img_x = Image.open(x_path)
        img_y = Image.open(y_path)
        if self.transform is not None:
            img_x = self.transform(img_x)
        if self.target_transform is not None:
            img_y = self.target_transform(img_y)
        return img_x, img_y

    def __len__(self):
        return len(self.imgs)

In [2]:
import torch.nn as nn
import torch
from torch import autograd


class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, input):
        return self.conv(input)


class Unet(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(Unet, self).__init__()

        self.conv1 = DoubleConv(in_ch, 64)
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = DoubleConv(64, 128)
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = DoubleConv(128, 256)
        self.pool3 = nn.MaxPool2d(2)
        self.conv4 = DoubleConv(256, 512)
        self.pool4 = nn.MaxPool2d(2)
        self.conv5 = DoubleConv(512, 1024)
        self.up6 = nn.ConvTranspose2d(1024, 512, 2, stride=2)
        self.conv6 = DoubleConv(1024, 512)
        self.up7 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.conv7 = DoubleConv(512, 256)
        self.up8 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.conv8 = DoubleConv(256, 128)
        self.up9 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.conv9 = DoubleConv(128, 64)
        self.conv10 = nn.Conv2d(64, out_ch, 1)

    def forward(self, x):
        c1 = self.conv1(x)
        p1 = self.pool1(c1)
        c2 = self.conv2(p1)
        p2 = self.pool2(c2)
        c3 = self.conv3(p2)
        p3 = self.pool3(c3)
        c4 = self.conv4(p3)
        p4 = self.pool4(c4)
        c5 = self.conv5(p4)
        up_6 = self.up6(c5)
        merge6 = torch.cat([up_6, c4], dim=1)
        c6 = self.conv6(merge6)
        up_7 = self.up7(c6)
        merge7 = torch.cat([up_7, c3], dim=1)
        c7 = self.conv7(merge7)
        up_8 = self.up8(c7)
        merge8 = torch.cat([up_8, c2], dim=1)
        c8 = self.conv8(merge8)
        up_9 = self.up9(c8)
        merge9 = torch.cat([up_9, c1], dim=1)
        c9 = self.conv9(merge9)
        c10 = self.conv10(c9)
        out = nn.Sigmoid()(c10)
        return out

In [36]:
# %matplotlib inline
# import matplotlib.pyplot as plt

# def imshow(inp, title=None):
#     """Imshow for Tensor."""
#     inp = inp.numpy().transpose((1, 2, 0))
#     mean = np.array([0.485, 0.456, 0.406])
#     std = np.array([0.229, 0.224, 0.225])
#     inp = std * inp + mean
#     inp = np.clip(inp, 0, 1)
#     plt.imshow(inp)
#     if title is not None:
#         plt.title(title)
#     plt.pause(0.001)

In [3]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch import autograd, optim
from torchvision.transforms import transforms

# 是否使用cuda

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 把多个步骤整合到一起, channel=（channel-mean）/std, 因为是分别对三个通道处理
x_transforms = transforms.Compose([
    transforms.ToTensor(),  # -> [0,1]
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # ->[-1,1]
])

# mask只需要转换为tensor
y_transforms = transforms.ToTensor()

def train_model(model, criterion, optimizer, dataload, num_epochs=20):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        dt_size = len(dataload.dataset)
        epoch_loss = 0
        step = 0
        for x, y in dataload:
            step += 1
            inputs = x.to(device)
            labels = y.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            print("%d/%d,train_loss:%0.3f" % (step, (dt_size - 1) // dataload.batch_size + 1, loss.item()))
        print("epoch %d loss:%0.3f" % (epoch, epoch_loss))
    torch.save(model.state_dict(), 'E:\\BaiduNetdiskDownload\\u_net_liver-master\\checkpoint\\weights_%d.pth' % epoch)
    return model


# 训练模型
def train(batch_size):
    model = Unet(3, 1).to(device)
    criterion = torch.nn.BCELoss()
    optimizer = optim.Adam(model.parameters())
    liver_dataset = LiverDataset("E:\\BaiduNetdiskDownload\\u_net_liver-master\\data\\train", transform=x_transforms, target_transform=y_transforms)
    dataloaders = DataLoader(liver_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    train_model(model, criterion, optimizer, dataloaders)


# 显示模型的输出结果
def test(model_dir):
    model = Unet(3, 1)
    model.load_state_dict(torch.load(model_dir, map_location='cpu'))
    liver_dataset = LiverDataset("E:\\BaiduNetdiskDownload\\u_net_liver-master\\data\\val", transform=x_transforms, target_transform=y_transforms)
    dataloaders = DataLoader(liver_dataset, batch_size=1)
    model.eval()
    import matplotlib.pyplot as plt
    plt.ion()
    with torch.no_grad():
        for x, _ in dataloaders:
            y = model(x)
            img_y = torch.squeeze(y).numpy()
            plt.imshow(img_y)
            plt.pause(0.1)
        plt.show()

In [4]:
train(2)

Epoch 0/19
----------
1/200,train_loss:0.680
2/200,train_loss:0.594
3/200,train_loss:0.511
4/200,train_loss:0.455
5/200,train_loss:0.435
6/200,train_loss:0.385
7/200,train_loss:0.377
8/200,train_loss:0.408
9/200,train_loss:0.369
10/200,train_loss:0.342
11/200,train_loss:0.387
12/200,train_loss:0.367
13/200,train_loss:0.309
14/200,train_loss:0.306
15/200,train_loss:0.306
16/200,train_loss:0.291
17/200,train_loss:0.289
18/200,train_loss:0.270
19/200,train_loss:0.267
20/200,train_loss:0.285
21/200,train_loss:0.247
22/200,train_loss:0.267
23/200,train_loss:0.245
24/200,train_loss:0.247
25/200,train_loss:0.247
26/200,train_loss:0.230
27/200,train_loss:0.250
28/200,train_loss:0.212
29/200,train_loss:0.207
30/200,train_loss:0.272
31/200,train_loss:0.198
32/200,train_loss:0.200
33/200,train_loss:0.238
34/200,train_loss:0.271
35/200,train_loss:0.195
36/200,train_loss:0.201
37/200,train_loss:0.202
38/200,train_loss:0.189
39/200,train_loss:0.180
40/200,train_loss:0.200
41/200,train_loss:0.215
42/

135/200,train_loss:0.051
136/200,train_loss:0.042
137/200,train_loss:0.035
138/200,train_loss:0.035
139/200,train_loss:0.028
140/200,train_loss:0.028
141/200,train_loss:0.032
142/200,train_loss:0.033
143/200,train_loss:0.048
144/200,train_loss:0.031
145/200,train_loss:0.023
146/200,train_loss:0.022
147/200,train_loss:0.041
148/200,train_loss:0.027
149/200,train_loss:0.043
150/200,train_loss:0.026
151/200,train_loss:0.027
152/200,train_loss:0.036
153/200,train_loss:0.025
154/200,train_loss:0.048
155/200,train_loss:0.022
156/200,train_loss:0.027
157/200,train_loss:0.022
158/200,train_loss:0.022
159/200,train_loss:0.019
160/200,train_loss:0.026
161/200,train_loss:0.027
162/200,train_loss:0.018
163/200,train_loss:0.033
164/200,train_loss:0.025
165/200,train_loss:0.023
166/200,train_loss:0.041
167/200,train_loss:0.028
168/200,train_loss:0.030
169/200,train_loss:0.019
170/200,train_loss:0.027
171/200,train_loss:0.021
172/200,train_loss:0.025
173/200,train_loss:0.032
174/200,train_loss:0.034


67/200,train_loss:0.017
68/200,train_loss:0.016
69/200,train_loss:0.018
70/200,train_loss:0.021
71/200,train_loss:0.040
72/200,train_loss:0.070
73/200,train_loss:0.031
74/200,train_loss:0.021
75/200,train_loss:0.031
76/200,train_loss:0.032
77/200,train_loss:0.045
78/200,train_loss:0.051
79/200,train_loss:0.027
80/200,train_loss:0.022
81/200,train_loss:0.057
82/200,train_loss:0.022
83/200,train_loss:0.015
84/200,train_loss:0.050
85/200,train_loss:0.040
86/200,train_loss:0.018
87/200,train_loss:0.022
88/200,train_loss:0.028
89/200,train_loss:0.032
90/200,train_loss:0.018
91/200,train_loss:0.040
92/200,train_loss:0.013
93/200,train_loss:0.015
94/200,train_loss:0.016
95/200,train_loss:0.038
96/200,train_loss:0.016
97/200,train_loss:0.026
98/200,train_loss:0.014
99/200,train_loss:0.018
100/200,train_loss:0.016
101/200,train_loss:0.016
102/200,train_loss:0.024
103/200,train_loss:0.032
104/200,train_loss:0.017
105/200,train_loss:0.025
106/200,train_loss:0.014
107/200,train_loss:0.013
108/200,

199/200,train_loss:0.010
200/200,train_loss:0.043
epoch 4 loss:2.662
Epoch 5/19
----------
1/200,train_loss:0.016
2/200,train_loss:0.017
3/200,train_loss:0.014
4/200,train_loss:0.013
5/200,train_loss:0.017
6/200,train_loss:0.015
7/200,train_loss:0.012
8/200,train_loss:0.015
9/200,train_loss:0.026
10/200,train_loss:0.016
11/200,train_loss:0.021
12/200,train_loss:0.010
13/200,train_loss:0.011
14/200,train_loss:0.011
15/200,train_loss:0.009
16/200,train_loss:0.017
17/200,train_loss:0.016
18/200,train_loss:0.012
19/200,train_loss:0.015
20/200,train_loss:0.012
21/200,train_loss:0.010
22/200,train_loss:0.008
23/200,train_loss:0.006
24/200,train_loss:0.008
25/200,train_loss:0.008
26/200,train_loss:0.014
27/200,train_loss:0.017
28/200,train_loss:0.012
29/200,train_loss:0.009
30/200,train_loss:0.012
31/200,train_loss:0.011
32/200,train_loss:0.012
33/200,train_loss:0.012
34/200,train_loss:0.013
35/200,train_loss:0.009
36/200,train_loss:0.009
37/200,train_loss:0.004
38/200,train_loss:0.012
39/200

133/200,train_loss:0.010
134/200,train_loss:0.009
135/200,train_loss:0.012
136/200,train_loss:0.008
137/200,train_loss:0.012
138/200,train_loss:0.013
139/200,train_loss:0.019
140/200,train_loss:0.009
141/200,train_loss:0.015
142/200,train_loss:0.009
143/200,train_loss:0.011
144/200,train_loss:0.013
145/200,train_loss:0.012
146/200,train_loss:0.010
147/200,train_loss:0.009
148/200,train_loss:0.010
149/200,train_loss:0.010
150/200,train_loss:0.009
151/200,train_loss:0.008
152/200,train_loss:0.007
153/200,train_loss:0.006
154/200,train_loss:0.016
155/200,train_loss:0.008
156/200,train_loss:0.009
157/200,train_loss:0.017
158/200,train_loss:0.013
159/200,train_loss:0.011
160/200,train_loss:0.009
161/200,train_loss:0.010
162/200,train_loss:0.010
163/200,train_loss:0.014
164/200,train_loss:0.013
165/200,train_loss:0.009
166/200,train_loss:0.012
167/200,train_loss:0.008
168/200,train_loss:0.008
169/200,train_loss:0.006
170/200,train_loss:0.009
171/200,train_loss:0.015
172/200,train_loss:0.007


KeyboardInterrupt: 

In [ ]:
# test()
model_dir = ""
test()